In [1]:
import torchvision
import torchvision.transforms as transforms
import torch
from torch.utils.data import TensorDataset, DataLoader, Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from albumentations import Compose, VerticalFlip, HorizontalFlip, Rotate, GridDistortion
import numpy as np
import os
import cv2
import glob

In [2]:
TRAIN_IMAGES = '/home/ghyeon/dataset/cloud/preprocess_images/train_images'
TEST_IMAGES = '/home/ghyeon/dataset/cloud/preprocess_images/test_images'
CSV_FILE = '/home/ghyeon/dataset/cloud/preprocess_images/350_525_train.csv'
w = 525
h = 350

In [3]:
train_df = pd.read_csv(CSV_FILE)
train_df.head()

,Image_Label,EncodedPixels
0,0011165.jpg_Fish,16530 235 16880 234 17230 234 17580 234 17930 ...
1,0011165.jpg_Flower,84792 250 85142 250 85492 250 85842 250 86192 ...
2,0011165.jpg_Gravel,NaN
3,0011165.jpg_Sugar,NaN
4,002be4f.jpg_Fish,14703 220 15053 220 15403 220 15753 220 16103 ...


In [4]:
train_df['ImageId'] = train_df['Image_Label'].apply(lambda x: x.split('_')[0])
train_df['ClassId'] = train_df['Image_Label'].apply(lambda x: x.split('_')[1])
train_df['hasMask'] = ~ train_df['EncodedPixels'].isna()

print(train_df.shape)
train_df.head()

(22184, 5)


,Image_Label,EncodedPixels,ImageId,ClassId,hasMask
0,0011165.jpg_Fish,16530 235 16880 234 17230 234 17580 234 17930 ...,0011165.jpg,Fish,True
1,0011165.jpg_Flower,84792 250 85142 250 85492 250 85842 250 86192 ...,0011165.jpg,Flower,True
2,0011165.jpg_Gravel,NaN,0011165.jpg,Gravel,False
3,0011165.jpg_Sugar,NaN,0011165.jpg,Sugar,False
4,002be4f.jpg_Fish,14703 220 15053 220 15403 220 15753 220 16103 ...,002be4f.jpg,Fish,True


In [5]:
mask_count_df = train_df.groupby('ImageId').agg(np.sum).reset_index()
mask_count_df.sort_values('hasMask', ascending=False, inplace=True)
print(mask_count_df.shape)
mask_count_df.head()

(5546, 2)


,ImageId,hasMask
821,24dd99c.jpg,4.0
1885,562f80a.jpg,4.0
3260,944f10b.jpg,4.0
1872,55b539f.jpg,4.0
4464,cdf7242.jpg,4.0


In [6]:
def rle2mask(rle, input_shape):
    width, height = input_shape[:2]
    
    mask = np.zeros(width * height).astype(np.uint8)
    
    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]
    lengths = array[1::2]

    current_position = 0
    for index, start in enumerate(starts):
        mask[int(start):int(start + lengths[index])] = 1
        current_position += lengths[index]
    current_output = mask.reshape(height, width).T
    return current_output

def build_masks(rles, input_shape, reshape=None):
    depth = len(rles)
    d = depth
    if reshape is None:
        masks = np.zeros((*input_shape, depth))
    else:
        masks = np.zeros((*reshape, depth))
    
    for i, rle in enumerate(rles):
        if type(rle) is str:
            if reshape is None:
                masks[:, :, i] = rle2mask(rle, input_shape)
            else:
                mask = rle2mask(rle, input_shape)
                reshaped_mask = np_resize(mask, reshape)
                masks[:, :, i] = reshaped_mask
    
    return masks

def build_rles(masks, reshape=None):
    width, height, depth = masks.shape
    
    rles = []
    
    for i in range(depth):
        mask = masks[:, :, i]
        
        if reshape:
            mask = mask.astype(np.float32)
            mask = np_resize(mask, reshape).astype(np.int64)
        
        rle = mask2rle(mask)
        rles.append(rle)
        
    return rles

In [7]:
class CloudDataset(Dataset):
    def __init__(self, list_IDs, df, images_list=None, target_df=None, batch_size=8, mode='train', shuffle=True, augmentation=None, resized_height=350, resized_width=525, num_channels=3, dim=(350, 525, 3), random_state=2019, base_path='/home/ghyeon/dataset/cloud/preprocess_images/train_images', reshape=None):
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.augmentation = augmentation
        if images_list is None:
            self.images_list = os.listdir(images_list)
        else:
            self.images_list = images_list
#         self.image_folder = image_folder
        self.dim = dim
        self.df = df
        self.base_path = base_path
        self.target_df = target_df
        self.reshape = reshape
        self.length = len(self.images_list) // self.batch_size
        self.resized_height = resized_height
        self.resized_width = resized_width
        self.num_channels = num_channels
        self.num_classes = 4
        self.list_IDs = list_IDs
        self.is_test = mode != 'train'
        self.random_state = random_state
#         if not shuffle and not self.is_test:
#             self.labels = [img_2_ohe_vector[img] for img in self.images_list[:self.length*self.batch_size]]
        self.on_epoch_end()
            
    def on_epoch_start(self):
        if self.shuffle:
            random.shuffle(self.images_list)
            
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.seed(self.random_state)
            np.random.shuffle(self.indexes)
            
    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_batch = [self.list_IDs[k] for k in indexes]
        
        X = self.__generate_X(list_IDs_batch)
        
        if self.is_test:
            return X
        else:
            y = self.__generate_y(list_IDs_batch)
            return X, y
    
    def __generate_X(self, list_IDs_batch):
        'Generates data containing batch_size samples'
        # Initialization
#         if self.reshape is None:
        X = np.empty((self.batch_size, *self.dim[:-1], self.num_channels))
#         else:
#             X = np.empty((self.batch_size, *self.reshape, self.n_channels))
        
        # Generate data
        for i, ID in enumerate(list_IDs_batch):
            im_name = self.df['ImageId'].iloc[ID]
            img_path = f"{self.base_path}/{im_name}"
            #img = self.__load_grayscale(img_path)
            #img = cv2.imread(img_path)
            im = glob.glob(img_path)
            for j in range(len(im)):
                img = cv2.imread(im[j])
                img = cv2.resize(img,(w,h))
    
       
            #img = image.load_img(img_path, target_size=(img_size, img_size))
            # Preprocessing the image
            #img = image.img_to_array(img)
            #img = np.expand_dims(img, axis=0)
            
            #img = cv2.resize(img, dim, interpolation = cv2.INTER_LINEAR)
            # Store samples
            X[i,] = img
            #print(X)

        return X
    
    def __generate_y(self, list_IDs_batch):
#         if self.reshape is None:
        y = np.empty((self.batch_size, *self.dim[:-1], self.num_classes), dtype=int)
#         else:
#             y = np.empty((self.batch_size, *self.reshape, self.n_classes), dtype=int)
        
        for i, ID in enumerate(list_IDs_batch):
            im_name = self.df['ImageId'].iloc[ID]
            image_df = self.target_df[self.target_df['ImageId'] == im_name]
            
            rles = image_df['EncodedPixels'].values
            if self.reshape is not None:
                masks = build_masks(rles, input_shape=self.dim[:-1], reshape=self.reshape)
            else:
                masks = build_masks(rles, input_shape=self.dim[:-1])
            
            y[i,] = masks

        return y
    
    def __len__(self):
        return self.length
    
    def get_labels(self):
        if self.shuffle:
            images_current = self.images_list[:self.length*self.batch_size]
            labels = [img_2_ohe_vector[img] for img in images_current]
        else:
            labels = self.labels
        return np.array(labels)

In [8]:
train_idx, val_idx = train_test_split(
    mask_count_df.index, random_state=2019, test_size=0.2
)

In [9]:
albumentations_train = Compose([
    VerticalFlip(), HorizontalFlip(), Rotate(limit=20), GridDistortion()
], p=1)

In [10]:
data_generator_train = CloudDataset(train_idx, df=mask_count_df, target_df=train_df,augmentation=albumentations_train)
# data_generator_train_eval = CloudDataset(train_imgs, shuffle=False)
data_generator_val = CloudDataset(val_idx, df=mask_count_df, target_df=train_df, mode='val',shuffle=False)

In [14]:
data_generator_train.__getitem__(3)

(array([[[[ 62.,  55.,  36.],
          [ 83.,  75.,  58.],
          [183., 174., 161.],
          ...,
          [124., 115., 105.],
          [114., 104.,  94.],
          [ 86.,  74.,  64.]],
 
         [[ 53.,  43.,  25.],
          [ 58.,  48.,  31.],
          [ 56.,  44.,  32.],
          ...,
          [161., 152., 142.],
          [150., 140., 130.],
          [ 45.,  33.,  23.]],
 
         [[ 56.,  43.,  27.],
          [ 80.,  67.,  53.],
          [167., 153., 141.],
          ...,
          [125., 116., 107.],
          [153., 142., 134.],
          [ 65.,  53.,  43.]],
 
         ...,
 
         [[160., 155., 154.],
          [180., 175., 172.],
          [116., 107., 103.],
          ...,
          [ 31.,  17.,  11.],
          [106.,  92.,  86.],
          [ 36.,  22.,  16.]],
 
         [[195., 187., 187.],
          [ 99.,  87.,  85.],
          [ 30.,  16.,  10.],
          ...,
          [ 63.,  48.,  45.],
          [ 87.,  75.,  71.],
          [ 82.,  67.,  64.